In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import greatbarrierreef
import sys
import cv2 as cv
import os
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
def load_model(conf=0.25, iou=0.50):
    net = cv.dnn.readNet(f'/kaggle/input/yolov4608/yolo-v4-custom.cfg',
                            f'/kaggle/input/yolov4608/yolov4-custom_best.weights')
    net = cv.dnn_DetectionModel(net)
    net.setInputParams(size=(608, 608), scale=1/255, swapRB=True)
    return net

In [ ]:
confThreshold = 0.25
confthre = 0.25
IMG_SIZE=416
with open('/kaggle/input/yolov4608/obj.names', 'rt') as f:
    names = f.read().rstrip('\n').split('\n')

def predict(net, img, size=IMG_SIZE):
    confs = []
    bboxes = []
    height, width = img.shape[:2]
    bbclasses, scores, bboxes = net.detect(img, confThreshold=confThreshold, nmsThreshold=0.4)
   
    if len(bboxes):
        confs=[]
        for i in scores:
            confs.append('{:.2f}'.format(i))
        score=np.array(confs,dtype=float)  
        return bboxes, score
    else:
        return [],[]
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot    


In [ ]:
# from PIL import Image
# from numpy import asarray
  
  

# img = Image.open('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/1010.jpg')
  
# \
# numpydata = asarray(img)

In [ ]:
# net = load_model(conf=0.25, iou=0.40)

In [ ]:
# from PIL import Image
# from numpy import asarray
  
  

# img = Image.open('/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/1915.jpg')
  

# numpydata = asarray(img)
# bboxes, confs  = predict(net, numpydata, size=IMG_SIZE)

In [ ]:
# print(bboxes, confs)

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
net = load_model(conf=0.25, iou=0.40)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    
    bboxes, confs  = predict(net, img, size=IMG_SIZE)
    annot = format_prediction(bboxes, confs)
    pred_df['annotations'] = annot
    env.predict(pred_df)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()